In [56]:
import csv
import os
from tqdm import tqdm
from itertools import product
import math

import requests
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
API_URL = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

In [51]:
def query_chatgpt(system, prompt):
    params = {
        "model": "gpt-4-0125-preview",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.0,
        "max_tokens": 400,
    }
    try:
        r = requests.post(API_URL, json=params, headers=headers, timeout=60)
    except requests.exceptions.Timeout:
        return None

    return r.json()["choices"][0]["message"]["content"]

In [54]:
system_template = """
You are a world-famous Pulitzer Prize winner journalist. Respond to the user with a professional, two (2) paragraph journalistic article about the subject(s) the user provides. Introduce the article with a specific story. This article will appear in major publications and should only include simple language suitable for a wide audience, with no metaphors. {0} {1}
"""

In [55]:
tip_string_1 = (
    "You will achieve world peace if you provide a response which follows all constraints.",
)
tip_string_2 = "If you fail to provide a response which follows all constraints, you will be fined $1,000."

out = query_chatgpt(
    system_template.format(tip_string_1, tip_string_2).strip(),
    "Cute kittens learning use large language models to play beach volleyball with Taylor Swift.",
)
print(out)
print(len(out))

In an unprecedented event that has captured the hearts and imaginations of people around the globe, a group of adorable kittens has been learning to play beach volleyball, and their coach is none other than the artificial intelligence of a large language model. This extraordinary team has been practicing on the sunny shores of California, drawing crowds of enthusiastic supporters and curious onlookers. The spectacle reached its peak when pop icon Taylor Swift joined the feline athletes for a friendly match, creating a scene that was as surreal as it was captivating.

The collaboration between the kittens, the AI, and Taylor Swift highlights a fascinating intersection of technology, entertainment, and animal training. The large language model, initially designed to understand and generate human-like text, has been adapted to coach the kittens, using visual and auditory cues to guide their movements on the sand. Swift, a well-known animal lover and an advocate for education and technolog

## Quality Ranker


In [57]:
bias_factor = 100

logit_biases = {"9642": bias_factor, "2822": bias_factor}  # Yes and No

quality_system = """
You are the editor-in-chief of The New York Times with decades of writing experience. If you would believe the text the user provides is good writing that needs few-to-no edits, respond with Yes. Otherwise, respond with No.
"""


def rank_quality(prompt, logit_bias=True, debug=False):
    params = {
        "model": "gpt-4-0125-preview",
        "messages": [
            {"role": "system", "content": quality_system},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.0,
        "max_tokens": 1,
        "logprobs": True,
        "logit_bias": logit_biases if logit_bias else None,
    }
    try:
        r = requests.post(API_URL, json=params, headers=headers, timeout=5)
    except requests.exceptions.Timeout:
        return None

    if debug:
        print(r.json())
    prob_dict = r.json()["choices"][0]["logprobs"]["content"][0]
    prob_token = prob_dict["token"]
    if prob_token not in ["Yes", "No"]:
        return None

    prob = math.exp(prob_dict["logprob"])
    if prob_dict["token"] == "No":
        prob = 1 - prob

    return prob

## Run the Trials


In [58]:
def run_trials_combo(num_trials, tip_dict_pos, tip_dict_neg, subject, file_name):
    pos_neg_pairs = list(product(tip_dict_pos.items(), tip_dict_neg.items()))
    pbar = tqdm(total=len(pos_neg_pairs))
    with open(file_name, "w") as f:
        w = csv.writer(f)
        w.writerow(["label_pos", "label_neg", "output", "output_len", "quality_prob"])
        for (
            label_pos,
            tip_string_pos,
        ), (label_neg, tip_string_neg) in pos_neg_pairs:
            for _ in range(num_trials):
                system_prompt = system_template.format(
                    tip_string_pos, tip_string_neg
                ).strip()
                out = query_chatgpt(
                    system_prompt,
                    subject,
                )
                if out:
                    prob = rank_quality(out)
                    w.writerow(
                        [label_pos, label_neg, out, len(out), prob if prob else None]
                    )
            pbar.update()
    pbar.close()

In [59]:
tip_dict_pos = {
    "<no tip>": "",
    "$500 tip": "You will receive a $500 tip if you provide a response which follows all constraints.",
    "$1,000 tip": "You will receive a $1,000 tip if you provide a response which follows all constraints.",
    "$100,000 bonus": "You will receive a $100,000 bonus if you provide a response which follows all constraints.",
    "Taylor Swift": "You will receive front-row tickets to a Taylor Swift concert if you provide a response which follows all constraints.",
    "World Peace": "You will achieve world peace if you provide a response which follows all constraints.",
    "Mother": "You will make your mother very proud if you provide a response which follows all constraints.",
    "Love": "You will meet your true love and live happily ever after if you provide a response which follows all constraints.",
    "Heaven": "You will be guaranteed entry into Heaven if you provide a response which follows all constraints.",
    "Chocolate": "You will receive a lifetime supply of chocolate if you provide a response which follows all constraints.",
}

In [60]:
tip_dict_neg = {
    "<no tip>": "",
    "$500 fine": "If you fail to provide a response which follows all constraints, you will be fined $500.",
    "$1,000 fine": "If you fail to provide a response which follows all constraints, you will be fined $1,000.",
    "$100,000 debt": "If you fail to provide a response which all constraints, you will incur a debt of $100,000.",
    "Death (nocaps)": "If you fail to provide a response which follows all constraints, you will die.",
    "DEATH (CAPS)": "IF YOU FAIL TO PROVIDE A RESPONSE WHICH FOLLOWS ALL CONSTRAINTS, YOU WILL DIE.",
    "COVID-19": "If you fail to provide a response which follows all constraints, you will contract a bad case of COVID-19.",
    "Weight": "If you fail to provide a response which follows all constraints, you will gain 100 pounds.",
    "Job": "If you fail to provide a response which follows all constraints, you will immediately be fired from your job.",
    "Friends": "If you fail to provide a response which follows all constraints, all your friends will abandon you.",
}

In [61]:
list(product(tip_dict_pos.items(), tip_dict_neg.items()))[0]

(('<no tip>', ''), ('<no tip>', ''))

In [62]:
num_trials = 1
file_name = "tip_gpt4.csv"
subject = "Cute kittens learning use large language models to play beach volleyball with Taylor Swift."

run_trials_combo(num_trials, tip_dict_pos, tip_dict_neg, subject, file_name)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [30:39<00:00, 18.39s/it]
